### Part 1 - Import de todas as bibliotecas necessárias 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Part 2 - Leitura do local dataframe + informações básicas

In [ ]:
df = pd.read_csv('RTA Dataset.csv')
pd.set_option('display.max_columns', None)

df_1 = df.copy()
df_2 = df.copy()
df_3 = df.copy()
df_4 = df.copy()

In [ ]:
df.describe(include = 'object')

In [ ]:
df.isnull().sum()   #verificar quantas colunas têm valores nulos

### Part 3 - pre processing - divisão de experiências

 *  exp 1 - distribuir a proporção dos valores nulos (>4000) pelos restantes valores, mantendo a proporção
 *  exp 2 - eliminar colunas com valores nulos > 3500
 *  exp 3 - substituir valores nulos por valores em que a coluna da gravidade é a mesma (tendo em conta a sua moda)
 *  exp 4 - aplicar método do KNN às colunas com valores nulos

Antes de repartir em experiências, vamos analisar o que significam algumas colunas que aparentam não ser tão importantes para a nossa análise, eliminando-as de qualquer experiência

In [ ]:
type_of_vehicle = df['Type_of_vehicle'].value_counts()    #ELIMINAR COM CERTEZA -> irrelevante
print(type_of_vehicle, "\n\n")

owner_of_vehicle = df['Owner_of_vehicle'].value_counts()  #ELIMINAR COM CERTEZA -> irrelevante
print(owner_of_vehicle, "\n\n")

service_year_of_vehicle = df['Service_year_of_vehicle'].value_counts()    #ELIMINAR COM CERTEZA -> muitos nans
print(service_year_of_vehicle, "\n\n")

defect_of_vehicle = df['Defect_of_vehicle'].value_counts()  #tem 7777 "no defect"
print(defect_of_vehicle, "\n\n")    #ELIMINAR -> DADOS CONFUSOS

area_accident_occured = df['Area_accident_occured'].value_counts() #ELIMINAR -> IRRELEVANTE
print(area_accident_occured, "\n\n")

road_allignment = df['Road_allignment'].value_counts()  #ELIMINAR -> muitos dados iguais -> +80%  
print(road_allignment, "\n\n")

pedestrian_movement = df['Pedestrian_movement'].value_counts()  #ELIMINAR -> dados sem sentido
print(pedestrian_movement, "\n\n")

fitness_of_casuality = df['Fitness_of_casuality'].value_counts()  #ELIMINAR -> faltam dados e muitos iguais
print(fitness_of_casuality, "\n\n")

work_of_causalty = df['Work_of_casuality'].value_counts()
print(work_of_causalty, "\n\n")

casualty_class = df['Casualty_class'].value_counts()  #ELIMINAR -> ver matriz de correlação
print(casualty_class, "\n\n")

Após a análise, eliminamos as colunas que afetam pouco a nossa saída, que, como sabemos, é a última coluna: "Gravidade do Acidente"

In [ ]:
colunas_a_eliminar = ['Type_of_vehicle', 'Owner_of_vehicle', 'Defect_of_vehicle', 'Area_accident_occured',
                      'Road_allignment', 'Pedestrian_movement', 'Fitness_of_casuality', 'Work_of_casuality', 'Time', 'Day_of_week']

df.drop(colunas_a_eliminar, axis = 1, inplace = True)

Agora, para uma melhor compreensão da tabela, vamos renomear as colunas

In [ ]:
#time, day_of_week, age_band_of_driver, sex_of_driver, educational_level, vehicle_driver_relation, driving_experience
#type_of_vehicle, owner_of_vehicle, service_year_of_vehicle, defect_of_vehicle, area_accident_occured
#lanes_or_medians, road_allignment, types_of_junction, road_surface_type, road_surface_conditions, ligh_conditions
#weather_conditions, type_of_collision, number_of_vehicles_involved, number_of_casualties, vehicle_movement
#casualty_class, sex_of_casualty, age_band_of_casualty, casualty_severity, work_of_casualty, fitness_of_casualty
#pedestrian_movement, cause_of_accident, accident_severity,
colunas_renomeadas = ['Faixa Etária', 'Género', 'Nível de Educação', 'Relação com o Veículo', 
                      'Experiência de Condução', 'Idade do Veículo', 'Situação de Faixa', 'Tipo de Cruzamento',
                      'Tipo de Estrada', 'Condições do Piso', 'Condições de Visibilidade',
                      'Condiçoes Meteorológicas', 'Tipo de Colisão', 'N.º Veículos Envolvidos',
                      'Número de Vítimas', 'Movimento do Veículo', 'Tipo de Vítima','Género da Vítima',
                      'Faixa Etária da Vítima', 'Gravidade da Vítima',
                      'Causa do Acidente', 'Gravidade do Acidente']

df.columns = colunas_renomeadas
df.head()

Vamos agora perceber quantos valores nulos existem e ainda realizar a substituição desses mesmos pela moda da coluna.

Neste passo, fomos apenar substituir os valores nulos cujas colunas tinham menos de 1000 valores "NaN", uma vez que afeta pouco a proporção dos valores, servindo de base a todas as experiências

In [ ]:
df.replace('na', pd.NA, inplace=True) # substituir "na" por um valor realmente nulo
df.isnull().sum()

In [ ]:
df['Nível de Educação'] = df['Nível de Educação'].fillna(df['Nível de Educação'].mode()[0])
df['Relação com o Veículo'] = df['Relação com o Veículo'].fillna(df['Relação com o Veículo'].mode()[0])
df['Experiência de Condução'] = df['Experiência de Condução'].fillna(df['Experiência de Condução'].mode()[0])
df['Situação de Faixa'] = df['Situação de Faixa'].fillna(df['Situação de Faixa'].mode()[0])
df['Tipo de Cruzamento'] = df['Tipo de Cruzamento'].fillna(df['Tipo de Cruzamento'].mode()[0])
df['Tipo de Estrada'] = df['Tipo de Estrada'].fillna(df['Tipo de Estrada'].mode()[0])
df['Tipo de Colisão'] = df['Tipo de Colisão'].fillna(df['Tipo de Colisão'].mode()[0])
df['Movimento do Veículo'] = df['Movimento do Veículo'].fillna(df['Movimento do Veículo'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
df_1.isnull().sum()

In [ ]:
df_1 = df.copy()
df_2 = df.copy()
df_3 = df.copy()
df_4 = df.copy()

#### Part 3.1 - distribuir a proporção dos valores nulos

In [ ]:
# definimos as colunas que queremos alterar
colunas_a_substituir = ['Idade do Veículo', 'Tipo de Vítima', 'Género da Vítima', 'Faixa Etária da Vítima', 'Gravidade da Vítima']

for coluna in colunas_a_substituir:

    no_valores_nulos = df_1[coluna].isnull().sum()

    # proporção de valores não nulos para cada categoria
    proporcoes_categoria = df_1[coluna].value_counts(normalize = True)

    # calcular o número de valores nulos a serem distribuídos para cada categoria
    distribuicao_nulos = (proporcoes_categoria * no_valores_nulos).round().astype(int)

    # neste caso, damos assign a cada valor nulo de forma RANDOM
    indices_nulo = df_1[df_1[coluna].isnull()].index
    for category, count in distribuicao_nulos.items():
        sample_indices = np.random.choice(indices_nulo, size = count, replace = False)
        df_1.loc[sample_indices, coluna] = category

    # preencher algum valor nulo que falte, RANDOM
    remaining_null_indices = df_1[df_1[coluna].isnull()].index
    remaining_categories = list(proporcoes_categoria.index)
    for index in remaining_null_indices:
        df_1.at[index, coluna] = np.random.choice(remaining_categories)

In [ ]:
df_1.isnull().sum()

In [ ]:
percentage_values = {}
for column in df_1.columns:
    counts = df_1[column].value_counts(normalize = True, dropna = False) * 100
    percentage_values[column] = counts

for column, percentages in percentage_values.items():
    print(percentages)
    print()

#### Part 3.2 - eliminar colunas com valores nulos > 3500

In [ ]:
colunas_valores_nulos = ['Idade do Veículo', 'Tipo de Vítima', 'Género da Vítima', 'Faixa Etária da Vítima', 'Gravidade da Vítima']

df_2.drop(colunas_valores_nulos, axis = 1, inplace = True)

In [ ]:
df_2.isnull().sum()

#### Part 3.3 - substituir valores nulos por valores em que a coluna da gravidade é a mesma (tendo em conta a sua moda)

In [ ]:
percentage_values = {}
for column in colunas_a_substituir:
    counts = df_3[column].value_counts(normalize = True, dropna = False) * 100
    percentage_values[column] = counts

for column, percentages in percentage_values.items():
    print(percentages)
    print()

In [ ]:
# Define the target column
target_column = 'Gravidade do Acidente'

# Iterate over each column with null values
for column in colunas_a_substituir:
    
    # Calculate the proportions of each value in the column for each category in the target column
    proportions = df_3.groupby(target_column)[column].value_counts(normalize=True)
    
    # For each null value in the column, replace it based on the proportions
    for index, row in df_3[df_3[column].isnull()].iterrows():
        
        # Get the proportions for the target value of this row
        target_value = row[target_column]
        target_proportions = proportions[target_value]
        
        # Sample from the proportions to replace the null value
        new_value = target_proportions.sample(weights=target_proportions).index[0]
        
        # Replace the null value
        df_3.at[index, column] = new_value

In [ ]:
df_3.isnull().sum()

In [ ]:
percentage_values = {}
for column in colunas_a_substituir:
    counts = df_3[column].value_counts(normalize = True, dropna = False) * 100
    percentage_values[column] = counts

for column, percentages in percentage_values.items():
    print(percentages)
    print()

#### Part 3.4 - aplicar método do KNN às colunas com valores nulos

In [25]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import numpy as np

# Initialize KNNImputer
imputer = KNNImputer()

# Get the columns with null values
columns_with_null = df_4.columns[df_4.isnull().any()]

# Iterate over each column with null values
for column in columns_with_null:
    # If the column is categorical, encode it using one-hot encoding or ordinal encoding
    if df_4[column].dtype == 'object':
        # Convert 'NAType' to NaN
        df_4[column] = df_4[column].replace('NAType', np.nan)
        # Handle missing values with a placeholder
        df_4[column] = df_4[column].fillna('missing')
        
        # Apply ordinal encoding
        encoder = OrdinalEncoder()
        encoded_values = encoder.fit_transform(df_4[[column]])
        encoded_df = pd.DataFrame(encoded_values, columns=[column+'_encoded'], index=df_4.index)
        # Drop the original categorical column and concatenate the encoded columns
        df_4 = pd.concat([df_4.drop(column, axis=1), encoded_df], axis=1)
    else:
        # Extract the column data
        X = df_4.dropna(subset=[column], axis=0).drop(columns_with_null, axis=1)
        y = df_4.dropna(subset=[column], axis=0)[column]
        X_with_null = df_4[df_4[column].isnull()].drop(columns_with_null, axis=1)
        
        # Apply KNN imputation
        imputer.fit(X, y)
        imputed_values = imputer.transform(X_with_null)
        
        # Update the dataframe with imputed values
        df_4.loc[X_with_null.index, column] = imputed_values


In [26]:
df_4.isnull().sum()

Faixa Etária                      0
Género                            0
Nível de Educação                 0
Relação com o Veículo             0
Experiência de Condução           0
Situação de Faixa                 0
Tipo de Cruzamento                0
Tipo de Estrada                   0
Condições do Piso                 0
Condições de Visibilidade         0
Condiçoes Meteorológicas          0
Tipo de Colisão                   0
N.º Veículos Envolvidos           0
Número de Vítimas                 0
Movimento do Veículo              0
Causa do Acidente                 0
Gravidade do Acidente             0
Idade do Veículo_1-2yr            0
Idade do Veículo_2-5yrs           0
Idade do Veículo_5-10yrs          0
Idade do Veículo_Above 10yr       0
Idade do Veículo_Below 1yr        0
Idade do Veículo_Unknown          0
Idade do Veículo_nan              0
Tipo de Vítima_encoded            0
Género da Vítima_encoded          0
Faixa Etária da Vítima_encoded    0
Gravidade da Vítima_encoded 

In [30]:
percentage_values = {}
for column in df_4.columns:
    counts = df_4[column].value_counts(normalize = True, dropna = False) * 100
    percentage_values[column] = counts

for column, percentages in percentage_values.items():
    print(percentages)
    print()

Faixa Etária
18-30       34.678467
31-50       33.184475
Over 51     12.869438
Unknown     12.569016
Under 18     6.698603
Name: proportion, dtype: float64

Género
Male       92.862943
Female      5.691783
Unknown     1.445274
Name: proportion, dtype: float64

Nível de Educação
Junior high school    67.879182
Elementary school     17.562520
High school            9.012666
Above high school      2.939266
Writing & reading      1.429035
Unknown                0.811952
Illiterate             0.365378
Name: proportion, dtype: float64

Relação com o Veículo
Employee    82.867814
Owner       16.019812
Other        0.998701
Unknown      0.113673
Name: proportion, dtype: float64

Experiência de Condução
5-10yr        34.037025
2-5yr         21.216304
Above 10yr    18.366353
1-2yr         14.257876
Below 1yr     10.896395
No Licence     0.958103
unknown        0.267944
Name: proportion, dtype: float64

Situação de Faixa
Two-way (divided with broken lines road marking)    38.941215
Undivided Two

# ELIMINAR AS COLUNAS 4000 E TRAZER ALGUMAS DE VOLTA QUE ELIMINÁMOS MAIS CEDO

# várias experiências (exp1 - manter proporção, exp2 - subs pela moda, exp3 - fazer o KNN, exp4 - eliminar colunas)

Ao analisar as 4 colunas com mais de 1000 valores "NA", temos 2 soluções: ou eliminamos as colunas, ou repartimos a sua proporção pelos restantes valores da coluna.

Eliminar uma coluna

Repartir a proporção pelos restantes valores das colunas

In [ ]:
# Define the columns to process
columns_to_process = ['Gravidade da Vítima', 'Faixa Etária da Vítima', 'Ocupação da Vítima']

for column in columns_to_process:
    # Number of null values in the column
    null_count = df[column].isnull().sum()

    # Proportion of non-null values for each category
    category_proportions = df[column].value_counts(normalize=True)

    # Calculate the number of null values to be distributed for each category
    null_distribution = (category_proportions * null_count).round().astype(int)

    # Randomly assign the null values to each category
    null_indices = df[df[column].isnull()].index
    for category, count in null_distribution.items():
        sample_indices = np.random.choice(null_indices, size=count, replace=False)
        df.loc[sample_indices, column] = category

    # Fill any remaining null values randomly
    remaining_null_indices = df[df[column].isnull()].index
    remaining_categories = list(category_proportions.index)
    for index in remaining_null_indices:
        df.at[index, column] = np.random.choice(remaining_categories)

Num ponto à parte, percebemos que eliminar as linhas com valores nulos não é uma opção, uma vez que perderíamos metade dos dados 

In [ ]:
no_linhas_na = df.isna().any(axis=1).sum()

print("Total de linhas com valores NaN", no_linhas_na)

### Part 5 - Fazer matriz de correlação e importância de variáveis

Primeiramente, vamos analisar a matriz de correlação entre as várias variáveis

In [ ]:
label_encoder = LabelEncoder()  # aplicar Label Encoder a todas as colunas "object"
for column in df_3.select_dtypes(include = 'object').columns:
    df_3[column] = label_encoder.fit_transform(df_3[column])

correlation_matrix = df_3.corr()
plt.figure(figsize = (20, 15))
sns.heatmap(correlation_matrix, annot = True, cmap = 'vlag', fmt = ".2f", linewidths = 1, square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df

In [ ]:
df_2 = df[['Gravidade da Vítima', 'Gravidade do Acidente']]
correlation_matrix = df_2.corr()
plt.figure(figsize = (10, 5))
sns.heatmap(correlation_matrix, annot = True, cmap = 'vlag', fmt = ".4f", linewidths = 1, square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
contingency_table = pd.crosstab(df['Gravidade do Acidente'], df['Faixa Etária da Vítima'], normalize='index') * 100

# Display the proportions
print(contingency_table)

De seguida, analisamos a importância de cada feature para o modelo 

In [ ]:
X = df.iloc[:, :-1].values # todas as features
Y =  df.iloc[:, -1].values # feature target

modelo = ExtraTreesClassifier()
modelo.fit(X, Y)

feature_importances = pd.Series(modelo.feature_importances_, index = df.columns[:-1])
feature_importances.nlargest(10).plot(kind='barh')  # mostrar as 10 features mais importantes

plt.show()

Neste ponto, dividimos os dados em sets de treino e de teste, utilizando 25% dos dados para teste e os restantes para teino

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 30)

print("Número de exemplos nos dados de treino: ", X_train.shape[0])
print("Número de exemplos nos dados de teste: ", X_test.shape[0])

### Parte 6 - Árvores de Decisão

O primeiro modelo de machine learning utilizado foi o modelo de árvores de decisão.


Primeiro vamos estanderizar as nossas features. Isso é feito para garantir que todas as características tenham a mesma escala, o que pode melhorar o desempenho do algoritmo. 

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# O parâmetro max_depth = 4 define a profundidade máxima da árvore de decisão, 
# visto que limitar a profundidade pode ajudar a evitar overfitting, tornando a árvore mais simples.
clf = DecisionTreeClassifier(random_state = 42, criterion = "entropy", max_depth = 4)

# Treino: Arvore de Decisão
clf = clf.fit(X_train, y_train)

# Previsão da resposta --> Teste
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

# Print dos valores accuracy
print('Train data accuracy: ', accuracy_score(y_true = y_train, y_pred = y_train_pred))
print('Test data accuracy: ', accuracy_score(y_true = y_test, y_pred = y_test_pred))
print('Decision tree accuracy: ', accuracy_score(y_test_pred, y_test))



In [ ]:
plt.figure(figsize = (20, 10))
tree.plot_tree(clf, fontsize = 8, feature_names = df.columns[:-1], filled = True, rounded = True, proportion=True)
plt.show()

Através do classification report, podemos concluir a acurácia do modelo tal como o recall, a precisão e o f1-score.

In [ ]:
print(classification_report(y_test, y_test_pred, zero_division = 1))

Após realizar um classification report, vamos agora mostrar a matriz de confusão. Assim podemos ver a quantidade de falsos positivos e falsos negativos.

In [ ]:
cf_matrix = confusion_matrix(y_test, y_test_pred)

ax = sns.heatmap(cf_matrix, annot = True, cmap = 'seismic', fmt = "4.0f")
ax.set_title('Seaborn Confusion Matrix\n\n')
ax.set_xlabel('\nPredicted decision case disposition')
ax.set_ylabel('Actual decision case disposition ')

print(cf_matrix)

plt.show()